# Graphentheorie

## 1. Graphen implementieren

Im folgenden wird eine Graphenrepräsentation in Phyton aufgebaut. 

Weder in Python noch in den meisten anderen Programmiersprachen existieren <i>generische</i> Datentypen, um Graphen zu modellieren. In einigen Programmiersprachen existieren allerdings heute Frameworks mit denen Graphen modelliert werden können. Programmiersprachen wie Julia und Scala profitieren von ihrem Abstraktionsgrad bei Graphen repräsentationen. Viele neue auf Graphen basierende Algorithmen werden deswegen in Scala oder Julia implementiert. 

Im Folgenden wird eine Graphimplementierung in Python vorgestellt. Dadurch soll aufgezeigt werden, wie Graphen in Programmiersprachen repräsentiert werden können ohne ein vorhandenes Framework dafür zu nutzen und wie eine Optimierungsproblem mit einfachen Algorithmen gelöst werden kann.

<b>Denke an unsere initiale Definition und vollziehe den Code nach:</b>

Ein Graph G besteht aus
- einer nichtleeren Knotenmenge V ; 
- und einer Kanten- oder Pfeilmenge E ⊆ V × V ;

bei der jedem Element aus E genau ein Knotenpaar i und j aus V zugeordnet wird.

<b>Knotenobjekt</b>

In [8]:
# Knotenobjekt das bisher nur den Namen repräsentiert

class Node(object):
    def __init__(self, name):
        """Annahme das name ein String ist"""
        self.name = name
        
    def getName(self):
        return self.name
    
    def __str__(self):
        return self.name

<b>Kantenobjekt</b>

In [14]:
# Kanten Object 
# das ungerichtet zwei knoten speichert, die die Kante verbindet
class Edge(object):
    def __init__(self, src, dest):  
        """Annahme dass src und dest Knoten sind""" 
        self.src = src 
        self.dest = dest   
        
    def getSource(self):
        return self.src 
    
    def getDestination(self): 
        return self.dest     
        
    def __str__(self): #ausgabe der Knoten die vom Kantenobjekt verbunden werden
        return self.src.getName() + '->' + self.dest.getName()

<b>Digraph</b>

In [41]:
#gerichteter Graph 
class Digraph(object): 
    """edges ist ein dict, welches alle Nachfloger zu einem Knoten speichert"""
    def __init__(self): 
        self.edges = {} 
        
    def addNode(self, node):
        if node in self.edges:
            raise ValueError('Duplicate node')
        else:
            self.edges[node] = []  
    
    def addEdge(self, edge):
        src = edge.getSource()
        dest = edge.getDestination()
        if not (src in self.edges and dest in self.edges):
            raise ValueError('Node not in graph')
        self.edges[src].append(dest)
        
    def childrenOf(self, node):
        return self.edges[node]
    
    def hasNode(self, node):
        return node in self.edges
    
    def getNode(self, name):
        for n in self.edges:
            if n.getName() == name:
                return n
        raise NameError(name)
        
    def __str__(self):
        result = ''
        for src in self.edges:
            for dest in self.edges[src]:
                result = result + src.getName() + '->'\
                         + dest.getName() + '\n'
        return result[:-1] #omit final newline

    

<b>Graph</b>

In [42]:
#ungerichteter Graph erbt 
class Graph(Digraph):
    def addEdge(self, edge):
        Digraph.addEdge(self, edge)
        rev = Edge(edge.getDestination(), edge.getSource())
        Digraph.addEdge(self, rev)

kleine Print Funktion

In [68]:
def printPath(path):
    """Annahmte Pfad ist eine Liste von Knoten"""
    result = ''
    for i in range(len(path)):
        result = result + str(path[i])
        if i != len(path) - 1:
            result = result + '->'
    return result 

<b>Bauen wir unseres ersten Graphen</b>

Im folgenden wir die implementierung oben gentutzt um einen Graphen anzulegen. Dabei handelt es sich, um mit Straßen verbunden Städte.

In [43]:
def buildCityGraph(graphType):
    g = graphType()
    for name in ('Boston', 'Providence', 'New York', 'Chicago',
                 'Denver', 'Phoenix', 'Los Angeles'): #Create 7 nodes
        g.addNode(Node(name))
    g.addEdge(Edge(g.getNode('Boston'), g.getNode('Providence')))
    g.addEdge(Edge(g.getNode('Boston'), g.getNode('New York')))
    g.addEdge(Edge(g.getNode('Providence'), g.getNode('Boston')))
    g.addEdge(Edge(g.getNode('Providence'), g.getNode('New York')))
    g.addEdge(Edge(g.getNode('New York'), g.getNode('Chicago')))
    g.addEdge(Edge(g.getNode('Chicago'), g.getNode('Phoenix')))
    g.addEdge(Edge(g.getNode('Chicago'), g.getNode('Denver')))
    g.addEdge(Edge(g.getNode('Denver'), g.getNode('Phoenix')))
    g.addEdge(Edge(g.getNode('Denver'), g.getNode('New York')))
    g.addEdge(Edge(g.getNode('Los Angeles'), g.getNode('Boston')))
    return g


In [45]:
g = buildCityGraph(Digraph)

In [47]:
# Alle Pfeile des Graphen anzeigen 
print(g)

Boston->Providence
Boston->New York
Providence->Boston
Providence->New York
New York->Chicago
Chicago->Phoenix
Chicago->Denver
Denver->Phoenix
Denver->New York
Los Angeles->Boston


<b>Aufgaben:</b>
- warum ist Graph eine Subklasse von Digraph?
- Zeichne den Graph auf ein Blatt Papier!

--> Platz für deine Antworten

## 2. Kürzeste Wege

Imfolgenden werden Suchalgorithmen vorgestellt, die dazu dienen eine Route im Graphen von einem Anfangspunkt zu einem Zielpunkt zu ermitteln. Suchalgorithmen

### 2.1 DFS

Als erstes implementieren wir eine Suche anhand des Deept First Algorithmus (DFS). Mit dem DFS läuft man also graphen ab, um die kürzeste Route zu einem Element zu finden.

In [61]:
def DFS(graph, start, end, path, shortest, toPrint = False):
    """Assumes graph is a Digraph; start and end are nodes;
          path and shortest are lists of nodes
       Returns a shortest path from start to end in graph"""
    path = path + [start]
    if toPrint:
        print('Current DFS path:', printPath(path))
    if start == end:
        return path
    for node in graph.childrenOf(start):
        if node not in path: #avoid cycles
            if shortest == None or len(path) < len(shortest):
                newPath = DFS(graph, node, end, path, shortest,
                              toPrint)
                if newPath != None:
                    shortest = newPath
        elif toPrint:
            print('Already visited', node)
    return shortest

#### Test des DFS
Test an unserem oben erstellten Graphen.

In [62]:
def shortestPath(graph, start, end, toPrint = False):
    """Assumes graph is a Digraph; start and end are nodes
       Returns a shortest path from start to end in graph"""
    return DFS(graph, start, end, [], None, toPrint)

In [63]:
def testSP(source, destination):
    g = buildCityGraph(Digraph)
    sp = shortestPath(g, g.getNode(source), g.getNode(destination),
                      toPrint = True)
    if sp != None:
        print('Shortest path from', source, 'to',
              destination, 'is', printPath(sp))
    else:
        print('There is no path from', source, 'to', destination)

#testSP('Chicago', 'Boston')
testSP('Boston', 'Phoenix')

Current DFS path: Boston
Current DFS path: Boston->Providence
Already visited Boston
Current DFS path: Boston->Providence->New York
Current DFS path: Boston->Providence->New York->Chicago
Current DFS path: Boston->Providence->New York->Chicago->Phoenix
Current DFS path: Boston->Providence->New York->Chicago->Denver
Already visited New York
Current DFS path: Boston->New York
Current DFS path: Boston->New York->Chicago
Current DFS path: Boston->New York->Chicago->Phoenix
Current DFS path: Boston->New York->Chicago->Denver
Already visited New York
Shortest path from Boston to Phoenix is Boston->New York->Chicago->Phoenix


<b>Aufgaben:</b>
    
- Vollziehe den Weg der DFS implementierung anhand deines oben gezeichneten Graphen nach
- Warum heißt der Algorithmus DFS / wie geht der Algorithmus vor?

--> Platz für deine Antworten

### 2.2 BFS

Als nächstes implementieren wir eine Suche anhand des Breadth-first-Search (BFS) Algorithmus. Mit dem BFS läuft man also ebenfalls graphen ab, um die kürzeste Route zu einem Element zu finden.

In [65]:
def BFS(graph, start, end, toPrint = False):
    """Assumes graph is a Digraph; start and end are nodes
       Returns a shortest path from start to end in graph"""
    initPath = [start]
    pathQueue = [initPath]
    while len(pathQueue) != 0:
        #Get and remove oldest element in pathQueue
        tmpPath = pathQueue.pop(0)
        if toPrint:
            print('Current BFS path:', printPath(tmpPath))
        lastNode = tmpPath[-1]
        if lastNode == end:
            return tmpPath
        for nextNode in graph.childrenOf(lastNode):
            if nextNode not in tmpPath:
                newPath = tmpPath + [nextNode]
                pathQueue.append(newPath)
    return None

#### Test des BFS
Test an unserem oben erstellten Graphen.

In [66]:
def shortestPath(graph, start, end, toPrint = False):
    """Assumes graph is a Digraph; start and end are nodes
       Returns a shortest path from start to end in graph"""
    return BFS(graph, start, end, toPrint)
    
testSP('Boston', 'Phoenix')

Current BFS path: Boston
Current BFS path: Boston->Providence
Current BFS path: Boston->New York
Current BFS path: Boston->Providence->New York
Current BFS path: Boston->New York->Chicago
Current BFS path: Boston->Providence->New York->Chicago
Current BFS path: Boston->New York->Chicago->Phoenix
Shortest path from Boston to Phoenix is Boston->New York->Chicago->Phoenix


<b>Aufgaben:</b>
    
- Vollziehe den Weg der VFS implementierung anhand deines gezeichneten Graphen nach
- Wie geht der Algorithmus vor?
- Was passiert mit der Verbindung Providence --> Boston?

- Was ist der Unterschied zwischen den Verfahren?


--> Platz für deine Antworten

## Recap

Also:
- bisher haben wir nur die Suche anhand nicht gewichteter Graphen implementiert. 
- Aus dem DFS lässt sich aber leicht eine optimierung bauen, anstatt der verbindungen die aufsummiert werden, müssten nur die Gewichte aufsummiert werden.

Was noch:
- Graphen sind cool
- Graphen sind insbesondere brauchbar um relationen zwischen Objekten zu repräsentieren
- Viele Probleme können mittels graphen modelliert werden und ein einfacher Algorithmus der sogar mit gerichteten und gewichteten Graphen umgehen kann (DFS) wurde oben vorgestellt.

### License

MIT License

Copyright (c) 2020 Fabian Gwinner - Julius-Maximilians-Universität Würzburg.

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

https://opensource.org/licenses/mit-license.php

Used Libraries are excluded und underlay their own Licenses